In [103]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
import re
from sklearn.metrics import *
from sklearn.feature_extraction.text import *
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
from gensim.models import Word2Vec
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)
%matplotlib inline

In [104]:
m = Mystem()
regex = re.compile("[А-ЯЁёа-я:=!\)\()A-z\_\%/|]+")

In [105]:
def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""

In [106]:
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

In [107]:
data = pd.read_json("train.json")
data.tail(2)

,id,text,label,extracted_part
1797,903507299,"ДОКУМЕНТАЦИЯ О ЗАКУПКЕ ЗАПРОС ПРЕДЛОЖЕНИЙ В ЭЛЕКТРОННОЙ ФОРМЕ на право заключения договора на оказание услуг по оценке эффективности функционирования Систем энергетического менеджмента (СЭнМ) ДЗО ПАО «Россети» и проведение сравнительного анализа ДЗО ПАО «Россети» по требованиям стандарта ISO 50001:2018 г. Москва 2022 год СОДЕРЖАНИЕ 2СОДЕРЖАНИЕ 4I. ОБЩИЕ УСЛОВИЯ ПРОВЕДЕНИЯ закупки 41. ОБЩИЕ ПОЛОЖЕНИЯ 41.1. Правовой статус документов 41.2. Заказчик, предмет и условия проведения закупки. 41.3. Начальная (максимальная) цена договора 51.4. Требования к участникам закупки 61.5. Привлечение соисполнителей (субподрядчиков) к исполнению договора 61.6. Расходы на участие в закупке и при заключении договора 61.7. Предоставление приоритетов товаров российского происхождения, работ, услуг, выполняемых, оказываемых российскими лицами, по отношению к товарам, происходящим из иностранного государства, работам, услугам, выполняемым, оказываемым иностранными лицами при проведении закупки 72. ДОКУМЕНТАЦИ 21",обеспечение гарантийных обязательств,"{'text': [''], 'answer_start': [0], 'answer_end': [0]}"
1798,926039184,"Государственная корпорацИя по атомной энергии «росатом» акционерное общество «НИИЭФА им. Д.В. Ефремова» (АО «НИИЭФА») УТВЕРЖДАЮ: Генеральный директор АО «НИИЭФА» ____________________ Е.А. Сакадынец «____» ______________ 2022г. ИЗВЕЩЕНИЕ О ПРОВЕДЕНИИ ЗАПРОСА КОТИРОВОК (ЗАКУПОЧНАЯ ДОКУМЕНТАЦИЯ) открытый запрос котировок в электронной форме на право заключения договора на поставку переносного рентгеновского аппарата направленного действия ICM SITE-X D1802 Извещение размещено на официальных сайтах http://www.zakupki.gov.ru, http://zakupki.rosatom.ru и http://roseltorg.ru/ от «07» сентября 2022 года № 220907/0459/___ Санкт-Петербург, 2022г. СОДЕРЖАНИЕ ИЗВЕЩЕНИЕ О ПРОВЕДЕНИИ ЗАПРОСА КОТИРОВОК 1 1. ИЗВЕЩЕНИЕ О ПРОВЕДЕНИИ ЗАКУПКИ 3 ЧАСТЬ 1 9 2. ТРЕБОВАНИЯ. ДОКУМЕНТЫ. СОСТАВ ЗАЯВКИ НА УЧАСТИЕ В ЗАКУПКЕ. 9 2.1. ТРЕБОВАНИЯ. ДОКУМЕНТЫ, ПОДТВЕРЖДАЮЩИЕ СООТВЕТСТВИЕ УСТАНОВЛЕННЫМ ТРЕБОВАНИЯМ. 9 2.1.1. Требования к участникам закупки 9 2.1.2. Требования к продукции 12 2.2. СОСТАВ ЗАЯВКИ НА УЧАСТИЕ В З ЧАСТЬ 4 Проект договора, который будет заключен по результатам закупки, приведен в виде отдельного файла в формате Word.",обеспечение гарантийных обязательств,"{'text': [''], 'answer_start': [0], 'answer_end': [0]}"


In [108]:
%time 
data.text = data.text.apply(words_only)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs


In [109]:
%time 
data.text = data.text.apply(lemmatize)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


In [110]:
data.head(2)

,id,text,label,extracted_part
0,809436509,извещение о проведение открытый конкурс в электронный форма для закупка общий информация номер извещение наименование объект закупка поставка продукт питание способ определение поставщик (подрядчик исполнитель) открытый конкурс в бль порядок внесение денежный средство в качество обеспечение заявка на участие в закупка а также условие гарантия обеспечение заявка на участие в закупка мочь предоставляться участник закупка в вид денежный средство или независимый гарантия предусматривать ст федеральный закон фз выбор способ обеспечение осуществляться участник закупка самостоятельно срок действие независимый гарантия должный составлять не мало месяц с дата окончание срок подача заявка обеспечение заявка на участие в закупка предоставляться в соответствие с ч ст федеральный закон фз условие независимый гарантия в соответствие со ст федеральный закон фз реквизит счет в соответствие с п ч ст закон фз номер расчетный счет номер лицевой счет см прилагать документ бик наименование кредитный организация номер корреспондентский счет обеспечение исполнение контракт требоваться обеспечение исполнение контракт размер обеспечение исполнение контракт российский рубль порядок обеспечение исполнение контракт требование к обеспечение исполнение контракт гарантийный обязательство мочь обеспечиваться предоставление независимый гарантия соответствующий требование ст федеральный закон фз или внесение денежный средство на указанный заказчик счет на который в соответствие с законодательство российский федерация учитываться операция со средство поступать заказчик способ обеспечение исполнение контракт гарантийный обязательство срок действие независимый гарантия определяться в соответствие с требование федеральный закон фз участник закупка с который заключаться контракт самостоятельно при это срок действие независимый гарантия должный превышать предусмотренный контракт срок исполнение обязательство который должный быть обеспечивать такой независимый гарантия не мало чем на один месяц в тот число в случай его изменение в соответствие со ст федеральный закон фз порядок предоставление и требование о информация и документ подтверждать соответствие участник закупка дополнительный требование: ) исполнять договор ) акт приемка оказывать услуга и (или) поставлять товар подтверждать цена оказывать услуга и (или) поставлять товар,обеспечение исполнения контракта,"{'text': ['Размер обеспечения исполнения контракта 6593.25 Российский рубль'], 'answer_start': [1279], 'answer_end': [1343]}"
1,854885310,требование к содержание заявка участник запрос котировка в электронный форма заявка на участие в запрос котировка в электронный форма должный содержать следующий информация и документ: ) об участник запрос котировка носить на банковский счет открывать в банк включать в перечень который утверждать распоряжение правительство российский федерация от N р требование к банк договор специальный счет к порядок использование иметься у участник закупка банковский счет в качество специальный счет правительство рф устанавливать в постановление от независимый гарантия должный соответствовать требование статья закон фз реквизит счет на который в соответствие с законодательство российский федерация учитываться операция со средство поступать заказчик: наименование заказчик: санкт петербургский государственный бюджетный учреждение здравоохранение городской поликлиника (спб гбуз ГП ) реквизит: инн КПП реквизит банковский счет: северо западный ГУ банк россия//УФК по г санкт петербург г санкт петербург бик номер счет банк получатель казначейский счет поставщик должный предоставлять обеспечение исполнение контракт в размер % от цена контракт в случай если обеспечение исполнение контракт предоставляться в вид передача заказчик денежный средство участник запрос котировка в электронный форма с который заключаться контракт перечислять денежный средство на счет: спб гбуз ГП инн КПП получатель платеж: комитет финансы спб ( спб гбуз ГП л/с ) реквизит банковс

In [111]:
data.shape

(1799, 4)

In [112]:
texts = [data.text.iloc[i].split() for i in range(len(data))]
texts[0]

['извещение',
 'о',
 'проведение',
 'открытый',
 'конкурс',
 'в',
 'электронный',
 'форма',
 'для',
 'закупка',
 'общий',
 'информация',
 'номер',
 'извещение',
 'наименование',
 'объект',
 'закупка',
 'поставка',
 'продукт',
 'питание',
 'способ',
 'определение',
 'поставщик',
 '(подрядчик',
 'исполнитель)',
 'открытый',
 'конкурс',
 'в',
 'бль',
 'порядок',
 'внесение',
 'денежный',
 'средство',
 'в',
 'качество',
 'обеспечение',
 'заявка',
 'на',
 'участие',
 'в',
 'закупка',
 'а',
 'также',
 'условие',
 'гарантия',
 'обеспечение',
 'заявка',
 'на',
 'участие',
 'в',
 'закупка',
 'мочь',
 'предоставляться',
 'участник',
 'закупка',
 'в',
 'вид',
 'денежный',
 'средство',
 'или',
 'независимый',
 'гарантия',
 'предусматривать',
 'ст',
 'федеральный',
 'закон',
 'фз',
 'выбор',
 'способ',
 'обеспечение',
 'осуществляться',
 'участник',
 'закупка',
 'самостоятельно',
 'срок',
 'действие',
 'независимый',
 'гарантия',
 'должный',
 'составлять',
 'не',
 'мало',
 'месяц',
 'с',
 'дата',
 

# Word2Vec модель

In [113]:
%%time
model = Word2Vec(texts, window=5, vector_size=300, min_count=5, workers=4)
model.save("word2v.model")

CPU times: user 7.92 s, sys: 74.3 ms, total: 7.99 s
Wall time: 6.7 s


Векторные представления получены, но для решения исходной задачи необходимо получить векторные представления для предложений. Для этого усредним векторы входящих в предложения слов. Тем самым, вектор предложения получится как средний вектор входящих в него слов

In [114]:
X = data.text.tolist()
y = data.label.tolist()

X, y = np.array(X), np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, 
                                                    random_state=1)
print ("Total train examples %s" % len(y_train))
print ("Total test examples %s" % len(y_test))

Total train examples 1205
Total test examples 594


# Mean Vector

In [115]:
w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))

In [116]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(w2v.popitem()[1])

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [117]:
rfc_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
    ("extra trees", RandomForestClassifier(n_estimators=200,
                                           max_depth=30,
                                           max_features=15,
                                           random_state=42,
                                           n_jobs=-1))])

In [118]:
rfc_w2v.fit(X_train,y_train)
pred = rfc_w2v.predict(X_test)

In [119]:
print("Precision: {0:6.2f}".format(precision_score(y_test, pred, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, pred, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, pred, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, pred)))
print(classification_report(y_test, pred))
labels = rfc_w2v.classes_

classification_report

Precision:   0.85
Recall:   0.85
F1-measure:   0.85
Accuracy:   0.85
                                      precision    recall  f1-score   support

обеспечение гарантийных обязательств       0.85      0.82      0.84       283
    обеспечение исполнения контракта       0.84      0.87      0.86       311

                            accuracy                           0.85       594
                           macro avg       0.85      0.85      0.85       594
                        weighted avg       0.85      0.85      0.85       594



<function sklearn.metrics._classification.classification_report(y_true, y_pred, *, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=False, zero_division='warn')>

# Средний вектор с весами tf-idf

Теперь воспользуемся более продвинутым методом получения эмбеддингов для предложений. Усредним вектора слов в предложении с весами Tf-Idf

In [120]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(w2v.popitem()[1])

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

# Классификация 

Теперь каждое слово представлено в виде вектора из 300 компонент. К таким данным мы можем применить любой ML алгоритм классификации. Например, RandomForest.

In [121]:
rfc_w2v = Pipeline([
    ("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)),
    ("extra trees", RandomForestClassifier(n_estimators=200,
                                           max_depth=30,
                                           max_features=15,
                                           random_state=42,
                                           n_jobs=-1))])

In [122]:
%%time
rfc_w2v.fit(X_train,y_train)
pred = rfc_w2v.predict(X_test)

CPU times: user 19.8 s, sys: 96.4 ms, total: 19.9 s
Wall time: 19.2 s


In [123]:
print("Precision: {0:6.2f}".format(precision_score(y_test, pred, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, pred, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, pred, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, pred)))
print(classification_report(y_test, pred))
labels = rfc_w2v.classes_

classification_report

Precision:   0.84
Recall:   0.84
F1-measure:   0.84
Accuracy:   0.84
                                      precision    recall  f1-score   support

обеспечение гарантийных обязательств       0.84      0.81      0.83       283
    обеспечение исполнения контракта       0.83      0.86      0.85       311

                            accuracy                           0.84       594
                           macro avg       0.84      0.84      0.84       594
                        weighted avg       0.84      0.84      0.84       594



<function sklearn.metrics._classification.classification_report(y_true, y_pred, *, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=False, zero_division='warn')>